# AI Search REST Data Pipeline
Azure AI Search를 위한 데이터 파이프라인 개발

## 1. 환경 설정 및 라이브러리 임포트

In [18]:
import os
import json
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Dict, List, Any

# Azure 관련 라이브러리
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient
from azure.cosmos import CosmosClient

# 데이터베이스 관련
import psycopg2
from sqlalchemy import create_engine, text

# HTTP 요청
import requests
from dotenv import load_dotenv

# 경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

print("라이브러리 임포트 완료")

라이브러리 임포트 완료


## 2. Azure AI Search 연결 설정

In [ ]:
# .env 파일 로드 (실제 사용시 .env 파일 생성 필요)
load_dotenv('.env')

# Azure AI Search 설정
AZURE_AI_SEARCH_ENDPOINT = os.getenv('AZURE_AI_SEARCH_ENDPOINT')
AZURE_AI_SEARCH_KEY = os.getenv('AZURE_AI_SEARCH_KEY')
AZURE_AI_SEARCH_INDEX = os.getenv('AZURE_AI_SEARCH_INDEX')

print(f"Endpoint: {AZURE_AI_SEARCH_ENDPOINT}")
print(f"Index: {AZURE_AI_SEARCH_INDEX}")
print(f"API Key: {'*' * 10 if AZURE_AI_SEARCH_KEY else 'Not set'}")  # 보안을 위해 마스킹

## 3. Azure AI Search 연결 테스트

In [20]:
def test_azure_search_connection():
    """Azure AI Search 연결 테스트"""
    
    # API 버전
    api_version = "2023-11-01"
    
    # 헤더 설정
    headers = {
        'Content-Type': 'application/json',
        'api-key': AZURE_AI_SEARCH_KEY
    }
    
    try:
        # 1. 서비스 통계 확인
        print("=" * 50)
        print("1. Azure AI Search 서비스 연결 테스트")
        print("=" * 50)
        
        stats_url = f"{AZURE_AI_SEARCH_ENDPOINT}/servicestats?api-version={api_version}"
        response = requests.get(stats_url, headers=headers)
        
        if response.status_code == 200:
            print("✅ 서비스 연결 성공!")
            stats = response.json()
            print(f"   - 사용 중인 인덱스 수: {stats.get('counters', {}).get('indexCounter', {}).get('usage', 'N/A')}")
            print(f"   - 최대 인덱스 수: {stats.get('counters', {}).get('indexCounter', {}).get('quota', 'N/A')}")
        else:
            print(f"❌ 서비스 연결 실패: {response.status_code}")
            print(f"   오류: {response.text}")
            return False
            
        # 2. 인덱스 정보 확인
        print("\n" + "=" * 50)
        print("2. 인덱스 정보 확인")
        print("=" * 50)
        
        index_url = f"{AZURE_AI_SEARCH_ENDPOINT}/indexes/{AZURE_AI_SEARCH_INDEX}?api-version={api_version}"
        response = requests.get(index_url, headers=headers)
        
        if response.status_code == 200:
            print(f"✅ 인덱스 '{AZURE_AI_SEARCH_INDEX}' 존재 확인!")
            index_info = response.json()
            print(f"   - 필드 수: {len(index_info.get('fields', []))}")
            print("   - 주요 필드:")
            for field in index_info.get('fields', [])[:5]:  # 처음 5개 필드만 표시
                print(f"     • {field['name']} ({field['type']})")
        elif response.status_code == 404:
            print(f"⚠️  인덱스 '{AZURE_AI_SEARCH_INDEX}'가 존재하지 않습니다.")
            print("   새 인덱스를 생성해야 합니다.")
        else:
            print(f"❌ 인덱스 확인 실패: {response.status_code}")
            print(f"   오류: {response.text}")
            
        # 3. 문서 수 확인
        print("\n" + "=" * 50)
        print("3. 인덱스 문서 수 확인")
        print("=" * 50)
        
        count_url = f"{AZURE_AI_SEARCH_ENDPOINT}/indexes/{AZURE_AI_SEARCH_INDEX}/docs/$count?api-version={api_version}"
        response = requests.get(count_url, headers=headers)
        
        if response.status_code == 200:
            doc_count = response.json()
            print(f"✅ 현재 인덱스에 저장된 문서 수: {doc_count}")
        elif response.status_code == 404:
            print("⚠️  인덱스가 존재하지 않아 문서 수를 확인할 수 없습니다.")
        else:
            print(f"❌ 문서 수 확인 실패: {response.status_code}")
            
        print("\n" + "=" * 50)
        print("테스트 완료!")
        print("=" * 50)
        return True
        
    except requests.exceptions.RequestException as e:
        print(f"❌ 연결 오류: {e}")
        return False
    except Exception as e:
        print(f"❌ 예상치 못한 오류: {e}")
        return False

# 테스트 실행
test_azure_search_connection()

1. Azure AI Search 서비스 연결 테스트
✅ 서비스 연결 성공!
   - 사용 중인 인덱스 수: N/A
   - 최대 인덱스 수: N/A

2. 인덱스 정보 확인
✅ 인덱스 'rag-1757403801556' 존재 확인!
   - 필드 수: 5
   - 주요 필드:
     • chunk_id (Edm.String)
     • parent_id (Edm.String)
     • chunk (Edm.String)
     • title (Edm.String)
     • text_vector (Collection(Edm.Single))

3. 인덱스 문서 수 확인
✅ 현재 인덱스에 저장된 문서 수: 38

테스트 완료!


True

## 4. 검색 테스트

In [21]:
def search_test(query="*", top=5):
    """Azure AI Search 검색 테스트
    
    Args:
        query: 검색 쿼리 (기본값: "*" - 모든 문서)
        top: 반환할 결과 수 (기본값: 5)
    """
    
    api_version = "2023-11-01"
    
    # 검색 요청 설정
    search_url = f"{AZURE_AI_SEARCH_ENDPOINT}/indexes/{AZURE_AI_SEARCH_INDEX}/docs/search?api-version={api_version}"
    
    headers = {
        'Content-Type': 'application/json',
        'api-key': AZURE_AI_SEARCH_KEY
    }
    
    # 검색 파라미터
    search_params = {
        "search": query,
        "top": top,
        "count": True,
        "select": "*"  # 모든 필드 반환
    }
    
    try:
        print("=" * 50)
        print(f"검색 쿼리: '{query}'")
        print("=" * 50)
        
        response = requests.post(search_url, json=search_params, headers=headers)
        
        if response.status_code == 200:
            results = response.json()
            total_count = results.get('@odata.count', 0)
            documents = results.get('value', [])
            
            print(f"✅ 검색 성공!")
            print(f"   총 결과 수: {total_count}")
            print(f"   반환된 문서 수: {len(documents)}")
            print("\n검색 결과:")
            print("-" * 50)
            
            if documents:
                for i, doc in enumerate(documents, 1):
                    print(f"\n문서 {i}:")
                    # 문서의 주요 필드만 표시 (처음 5개 필드)
                    for key, value in list(doc.items())[:5]:
                        if key.startswith('@'):  # 메타데이터 필드는 건너뛰기
                            continue
                        # 긴 텍스트는 축약
                        if isinstance(value, str) and len(value) > 100:
                            value = value[:100] + "..."
                        print(f"  - {key}: {value}")
            else:
                print("검색 결과가 없습니다.")
                
        elif response.status_code == 404:
            print(f"❌ 인덱스 '{AZURE_AI_SEARCH_INDEX}'를 찾을 수 없습니다.")
        else:
            print(f"❌ 검색 실패: {response.status_code}")
            print(f"   오류: {response.text}")
            
    except Exception as e:
        print(f"❌ 검색 중 오류 발생: {e}")

# 기본 검색 테스트 (모든 문서)
search_test()

# 특정 검색어로 테스트하려면:
# search_test("검색어", top=10)

검색 쿼리: '*'
✅ 검색 성공!
   총 결과 수: 38
   반환된 문서 수: 5

검색 결과:
--------------------------------------------------

문서 1:
  - chunk_id: 9177293483e9_aHR0cHM6Ly9kZXZzdG9yYWdlYWNjb3VudGRoc2VvLmJsb2IuY29yZS53aW5kb3dzLm5ldC9kZXYtc3RvcmFnZS1...
  - parent_id: aHR0cHM6Ly9kZXZzdG9yYWdlYWNjb3VudGRoc2VvLmJsb2IuY29yZS53aW5kb3dzLm5ldC9kZXYtc3RvcmFnZS1ibG9iLWNvbnRh...
  - chunk: <summary>번역수업</summary>

* stores: 저장
* Either 랑 or 가 맞는 조합
* Neither nor 가 맞는 조합
* Neither or 도 쓰긴 ...
  - title: exam1.md

문서 2:
  - chunk_id: 9177293483e9_aHR0cHM6Ly9kZXZzdG9yYWdlYWNjb3VudGRoc2VvLmJsb2IuY29yZS53aW5kb3dzLm5ldC9kZXYtc3RvcmFnZS1...
  - parent_id: aHR0cHM6Ly9kZXZzdG9yYWdlYWNjb3VudGRoc2VvLmJsb2IuY29yZS53aW5kb3dzLm5ldC9kZXYtc3RvcmFnZS1ibG9iLWNvbnRh...
  - chunk: Lake table. The **recent_sensor_recordings** table contains an identifying **sensor_id** alongside t...
  - title: exam1.md

문서 3:
  - chunk_id: 9177293483e9_aHR0cHM6Ly9kZXZzdG9yYWdlYWNjb3VudGRoc2VvLmJsb2IuY29yZS53aW5kb3dzLm5ldC9kZXYtc3RvcmFnZS1...
  - paren

## 5. Cosmos DB 연결 및 테스트

In [ ]:
# Cosmos DB 설정 로드
AZURE_COSMOS_ENDPOINT = os.getenv('AZURE_COSMOS_ENDPOINT')
AZURE_COSMOS_KEY = os.getenv('AZURE_COSMOS_KEY')
AZURE_COSMOS_DATABASE = os.getenv('AZURE_COSMOS_DATABASE')
AZURE_COSMOS_CONTAINER = os.getenv('AZURE_COSMOS_CONTAINER')

print(f"Cosmos DB Endpoint: {AZURE_COSMOS_ENDPOINT}")
print(f"Database: {AZURE_COSMOS_DATABASE}")
print(f"Container: {AZURE_COSMOS_CONTAINER}")
print(f"Key: {'*' * 10 if AZURE_COSMOS_KEY else 'Not set'}")  # 보안을 위해 마스킹

# Cosmos DB 클라이언트 초기화
from azure.cosmos import CosmosClient, PartitionKey, exceptions

cosmos_client = CosmosClient(AZURE_COSMOS_ENDPOINT, AZURE_COSMOS_KEY)
print("\n✅ Cosmos DB 클라이언트 초기화 완료")

In [23]:
def test_cosmos_db_connection():
    """Cosmos DB 연결 테스트"""
    
    print("=" * 60)
    print("Cosmos DB 연결 테스트")
    print("=" * 60)
    
    try:
        # 1. 데이터베이스 목록 확인
        databases = list(cosmos_client.list_databases())
        print(f"✅ Cosmos DB 연결 성공!")
        print(f"   현재 데이터베이스 수: {len(databases)}")
        print("   데이터베이스 목록:")
        for db in databases:
            print(f"     • {db['id']}")
        
        # 2. 특정 데이터베이스 확인
        try:
            database = cosmos_client.get_database_client(AZURE_COSMOS_DATABASE)
            properties = database.read()
            print(f"\n✅ 데이터베이스 '{AZURE_COSMOS_DATABASE}' 확인됨")
            
            # 컨테이너 목록 확인
            containers = list(database.list_containers())
            print(f"   컨테이너 수: {len(containers)}")
            if containers:
                print("   컨테이너 목록:")
                for container in containers:
                    print(f"     • {container['id']}")
        except exceptions.CosmosResourceNotFoundError:
            print(f"\n⚠️  데이터베이스 '{AZURE_COSMOS_DATABASE}'가 존재하지 않습니다.")
            
        return True
        
    except exceptions.CosmosHttpResponseError as e:
        print(f"❌ Cosmos DB 연결 실패: {e.message}")
        return False
    except Exception as e:
        print(f"❌ 예상치 못한 오류: {e}")
        return False

# 연결 테스트 실행
test_cosmos_db_connection()

Cosmos DB 연결 테스트
✅ Cosmos DB 연결 성공!
   현재 데이터베이스 수: 1
   데이터베이스 목록:
     • cosmos

✅ 데이터베이스 'cosmos' 확인됨
   컨테이너 수: 1
   컨테이너 목록:
     • cosmos-rubicon-001


True

In [24]:
def get_existing_database_and_container():
    """기존 데이터베이스 및 컨테이너 가져오기"""
    
    print("=" * 60)
    print("기존 데이터베이스 및 컨테이너 연결")
    print("=" * 60)
    
    try:
        # 1. 기존 데이터베이스 가져오기
        database = cosmos_client.get_database_client(AZURE_COSMOS_DATABASE)
        properties = database.read()
        print(f"✅ 데이터베이스 '{AZURE_COSMOS_DATABASE}' 연결됨")
        
        # 2. 기존 컨테이너 가져오기
        container = database.get_container_client(AZURE_COSMOS_CONTAINER)
        properties = container.read()
        print(f"✅ 컨테이너 '{AZURE_COSMOS_CONTAINER}' 연결됨")
        
        # 컨테이너 정보 표시
        print(f"\n📊 컨테이너 정보:")
        print(f"   ID: {properties['id']}")
        print(f"   파티션 키 경로: {properties['partitionKey']['paths']}")
        print(f"   인덱싱 정책: {properties['indexingPolicy']['indexingMode']}")
        
        # 현재 문서 수 확인
        query = "SELECT VALUE COUNT(1) FROM c"
        result = list(container.query_items(query=query, enable_cross_partition_query=True))
        doc_count = result[0] if result else 0
        print(f"   현재 문서 수: {doc_count}개")
        
        return database, container
        
    except exceptions.CosmosResourceNotFoundError as e:
        print(f"❌ 리소스를 찾을 수 없음: {e.message}")
        print(f"   데이터베이스 '{AZURE_COSMOS_DATABASE}' 또는 컨테이너 '{AZURE_COSMOS_CONTAINER}'가 존재하지 않습니다.")
        return None, None
    except exceptions.CosmosHttpResponseError as e:
        print(f"❌ 연결 실패: {e.message}")
        return None, None
    except Exception as e:
        print(f"❌ 예상치 못한 오류: {e}")
        return None, None

# 기존 데이터베이스 및 컨테이너 가져오기
database, container = get_existing_database_and_container()

기존 데이터베이스 및 컨테이너 연결
✅ 데이터베이스 'cosmos' 연결됨
✅ 컨테이너 'cosmos-rubicon-001' 연결됨

📊 컨테이너 정보:
   ID: cosmos-rubicon-001
   파티션 키 경로: ['/rubicon']
   인덱싱 정책: consistent
   현재 문서 수: 5개


In [25]:
def insert_sample_data(container):
    """샘플 데이터 삽입"""
    
    print("=" * 60)
    print("샘플 데이터 삽입")
    print("=" * 60)
    
    if not container:
        print("❌ 컨테이너가 초기화되지 않았습니다.")
        return []
    
    # 샘플 데이터 정의
    sample_data = [
        {
            "id": "doc1",
            "category": "metadata",
            "table_name": "customer_info",
            "table_description": "고객 정보 테이블",
            "columns": [
                {"name": "customer_id", "type": "VARCHAR", "description": "고객 고유 식별자"},
                {"name": "name", "type": "VARCHAR", "description": "고객 이름"},
                {"name": "email", "type": "VARCHAR", "description": "이메일 주소"},
                {"name": "registration_date", "type": "DATE", "description": "가입일"}
            ],
            "created_date": datetime.now().isoformat(),
            "tags": ["customer", "personal_info", "master_data"]
        },
        {
            "id": "doc2",
            "category": "metadata",
            "table_name": "order_history",
            "table_description": "주문 이력 테이블",
            "columns": [
                {"name": "order_id", "type": "BIGINT", "description": "주문 번호"},
                {"name": "customer_id", "type": "VARCHAR", "description": "고객 ID"},
                {"name": "order_date", "type": "TIMESTAMP", "description": "주문 시간"},
                {"name": "total_amount", "type": "DECIMAL", "description": "총 주문 금액"},
                {"name": "status", "type": "VARCHAR", "description": "주문 상태"}
            ],
            "created_date": datetime.now().isoformat(),
            "tags": ["order", "transaction", "history"]
        },
        {
            "id": "doc3",
            "category": "query_log",
            "query_id": "q001",
            "user_query": "최근 3개월 내 구매한 고객 목록 조회",
            "sql_query": "SELECT DISTINCT customer_id, name FROM customer_info WHERE customer_id IN (SELECT customer_id FROM order_history WHERE order_date >= DATEADD(month, -3, GETDATE()))",
            "execution_time_ms": 245,
            "result_count": 1542,
            "created_date": datetime.now().isoformat(),
            "tags": ["analytics", "customer_analysis"]
        },
        {
            "id": "doc4",
            "category": "data_profile",
            "table_name": "product_catalog",
            "row_count": 15234,
            "column_count": 12,
            "size_mb": 48.5,
            "last_updated": datetime.now().isoformat(),
            "quality_score": 0.92,
            "null_percentage": 3.2,
            "distinct_values": {
                "product_id": 15234,
                "category": 45,
                "brand": 127
            },
            "tags": ["catalog", "product", "master_data"]
        },
        {
            "id": "doc5",
            "category": "pipeline_log",
            "pipeline_name": "daily_data_sync",
            "run_id": "run_20250115_001",
            "status": "SUCCESS",
            "start_time": datetime.now().isoformat(),
            "end_time": datetime.now().isoformat(),
            "records_processed": 52341,
            "records_failed": 12,
            "error_messages": [],
            "tags": ["etl", "daily", "sync"]
        }
    ]
    
    inserted_items = []
    
    for item in sample_data:
        try:
            # 문서 삽입
            response = container.create_item(body=item)
            inserted_items.append(response)
            print(f"✅ 문서 삽입 성공: ID={item['id']}, Category={item['category']}")
        except exceptions.CosmosResourceExistsError:
            print(f"⚠️  문서 이미 존재: ID={item['id']}")
            # 기존 문서 업데이트
            try:
                response = container.replace_item(item=item['id'], body=item)
                inserted_items.append(response)
                print(f"   → 문서 업데이트됨")
            except Exception as e:
                print(f"   → 업데이트 실패: {e}")
        except Exception as e:
            print(f"❌ 문서 삽입 실패: ID={item['id']}, 오류={e}")
    
    print(f"\n📊 삽입 결과:")
    print(f"   총 시도: {len(sample_data)}개")
    print(f"   성공: {len(inserted_items)}개")
    
    return inserted_items

# 샘플 데이터 삽입
inserted_documents = insert_sample_data(container)

샘플 데이터 삽입
⚠️  문서 이미 존재: ID=doc1
   → 문서 업데이트됨
⚠️  문서 이미 존재: ID=doc2
   → 문서 업데이트됨
⚠️  문서 이미 존재: ID=doc3
   → 문서 업데이트됨
⚠️  문서 이미 존재: ID=doc4
   → 문서 업데이트됨
⚠️  문서 이미 존재: ID=doc5
   → 문서 업데이트됨

📊 삽입 결과:
   총 시도: 5개
   성공: 5개


In [31]:
def query_data(container):
    """데이터 조회 - 다양한 쿼리 예제"""
    
    print("=" * 60)
    print("데이터 조회")
    print("=" * 60)
    
    if not container:
        print("❌ 컨테이너가 초기화되지 않았습니다.")
        return
    
    # 1. 모든 문서 조회
    print("\n1️⃣ 모든 문서 조회")
    print("-" * 40)
    query = "SELECT * FROM c"
    items = list(container.query_items(query=query, enable_cross_partition_query=True))
    print(f"   총 문서 수: {len(items)}")
    for item in items:
        print(f"   • ID: {item['id']}, Category: {item.get('category', 'N/A')}")
    
    # 2. 특정 카테고리 필터링
    print("\n2️⃣ 특정 카테고리(metadata) 문서 조회")
    print("-" * 40)
    query = "SELECT c.id, c.table_name, c.table_description FROM c WHERE c.category = 'metadata'"
    items = list(container.query_items(query=query, enable_cross_partition_query=True))
    print(f"   메타데이터 문서 수: {len(items)}")
    for item in items:
        if 'table_name' in item and 'table_description' in item:
            print(f"   • {item['table_name']}: {item['table_description']}")
    
    # 3. 태그 기반 검색
    print("\n3️⃣ 특정 태그를 포함한 문서 조회")
    print("-" * 40)
    query = "SELECT c.id, c.category, c.tags FROM c WHERE ARRAY_CONTAINS(c.tags, 'customer')"
    try:
        items = list(container.query_items(query=query, enable_cross_partition_query=True))
        print(f"   'customer' 태그 문서 수: {len(items)}")
        for item in items:
            print(f"   • ID: {item['id']}, Tags: {item.get('tags', [])}")
    except Exception as e:
        print(f"   태그 검색 건너뛰기: {e}")
    
    # 4. 특정 ID로 단일 문서 조회
    print("\n4️⃣ 특정 ID로 단일 문서 조회")
    print("-" * 40)
    # 먼저 doc1이 어떤 파티션에 있는지 확인
    query = "SELECT c.id, c.category FROM c WHERE c.id = 'doc1'"
    items = list(container.query_items(query=query, enable_cross_partition_query=True))
    if items:
        doc_partition_key = items[0].get('category')
        try:
            item = container.read_item(item="doc1", partition_key=doc_partition_key)
            print(f"   문서 ID: {item['id']}")
            print(f"   카테고리: {item.get('category', 'N/A')}")
            if 'table_name' in item:
                print(f"   테이블명: {item['table_name']}")
            if 'columns' in item:
                print(f"   컬럼 수: {len(item['columns'])}")
        except exceptions.CosmosResourceNotFoundError:
            print("   문서를 찾을 수 없습니다.")
    else:
        print("   doc1 문서가 존재하지 않습니다.")
    
    # 5. 전체 문서 수 집계 (단순 COUNT)
    print("\n5️⃣ 전체 문서 수 집계")
    print("-" * 40)
    query = "SELECT VALUE COUNT(1) FROM c"
    result = list(container.query_items(query=query, enable_cross_partition_query=True))
    total_count = result[0] if result else 0
    print(f"   전체 문서 수: {total_count}개")
    
    # 카테고리별 개수는 클라이언트 측에서 집계
    print("\n   카테고리별 문서 수 (클라이언트 집계):")
    query = "SELECT c.category FROM c"
    items = list(container.query_items(query=query, enable_cross_partition_query=True))
    category_counts = {}
    for item in items:
        category = item.get('category', 'unknown')
        category_counts[category] = category_counts.get(category, 0) + 1
    for category, count in category_counts.items():
        print(f"   • {category}: {count}개")
    
    # 6. 조건부 필터링
    print("\n6️⃣ 조건부 필터링 (pipeline_log 상태가 SUCCESS인 문서)")
    print("-" * 40)
    query = "SELECT c.id, c.pipeline_name, c.status, c.records_processed FROM c WHERE c.category = 'pipeline_log' AND c.status = 'SUCCESS'"
    items = list(container.query_items(query=query, enable_cross_partition_query=True))
    if items:
        for item in items:
            print(f"   • Pipeline: {item.get('pipeline_name', 'N/A')}")
            print(f"     처리 레코드: {item.get('records_processed', 0):,}")
    else:
        print("   조건에 맞는 문서가 없습니다.")
    
    # 7. 프로젝션과 정렬
    print("\n7️⃣ 생성일 기준 정렬 (최신순)")
    print("-" * 40)
    query = "SELECT c.id, c.category, c.created_date FROM c WHERE IS_DEFINED(c.created_date) ORDER BY c.created_date DESC"
    try:
        items = list(container.query_items(query=query, enable_cross_partition_query=True))
        if items:
            for item in items[:3]:  # 상위 3개만 표시
                created_date = item.get('created_date', 'N/A')
                if created_date != 'N/A' and len(created_date) >= 19:
                    created_date = created_date[:19]
                print(f"   • ID: {item['id']}, Date: {created_date}")
        else:
            print("   created_date가 정의된 문서가 없습니다.")
    except Exception as e:
        print(f"   정렬 쿼리 실행 중 오류: {e}")

# 데이터 조회 실행
query_data(container)

데이터 조회

1️⃣ 모든 문서 조회
----------------------------------------
   총 문서 수: 5
   • ID: doc1, Category: metadata
   • ID: doc2, Category: metadata
   • ID: doc3, Category: query_log
   • ID: doc4, Category: data_profile
   • ID: doc5, Category: pipeline_log

2️⃣ 특정 카테고리(metadata) 문서 조회
----------------------------------------
   메타데이터 문서 수: 2
   • customer_info: 고객 정보 테이블
   • order_history: 주문 이력 테이블

3️⃣ 특정 태그를 포함한 문서 조회
----------------------------------------
   'customer' 태그 문서 수: 1
   • ID: doc1, Tags: ['customer', 'personal_info', 'master_data']

4️⃣ 특정 ID로 단일 문서 조회
----------------------------------------
   문서를 찾을 수 없습니다.

5️⃣ 전체 문서 수 집계
----------------------------------------
   전체 문서 수: 5개

   카테고리별 문서 수 (클라이언트 집계):
   • metadata: 2개
   • query_log: 1개
   • data_profile: 1개
   • pipeline_log: 1개

6️⃣ 조건부 필터링 (pipeline_log 상태가 SUCCESS인 문서)
----------------------------------------
   • Pipeline: daily_data_sync
     처리 레코드: 52,341

7️⃣ 생성일 기준 정렬 (최신순)
----------------------------

In [ ]:
def delete_data(container, delete_all=False):
    """데이터 삭제
    
    Args:
        container: Cosmos DB 컨테이너
        delete_all: True면 모든 문서 삭제, False면 선택적 삭제
    """
    
    print("=" * 60)
    print("데이터 삭제")
    print("=" * 60)
    
    if not container:
        print("❌ 컨테이너가 초기화되지 않았습니다.")
        return
    
    if delete_all:
        # 모든 문서 삭제
        print("\n⚠️  모든 문서 삭제 작업")
        print("-" * 40)
        
        # 먼저 모든 문서 조회
        query = "SELECT * FROM c"
        items = list(container.query_items(query=query, enable_cross_partition_query=True))
        
        if not items:
            print("   삭제할 문서가 없습니다.")
            return
        
        print(f"   삭제 대상: {len(items)}개 문서")
        
        deleted_count = 0
        failed_count = 0
        
        for item in items:
            try:
                partition_key_value = item.get('category')
                if partition_key_value is not None:
                    container.delete_item(item=item, partition_key=partition_key_value)
                    deleted_count += 1
                    print(f"   ✅ 삭제됨: ID={item['id']}")
                else:
                    print(f"   ⚠️  파티션 키 없음: ID={item['id']}")
                    failed_count += 1
            except Exception as e:
                failed_count += 1
                print(f"   ❌ 삭제 실패: ID={item['id']}, 오류={e}")
        
        print(f"\n📊 삭제 결과:")
        print(f"   성공: {deleted_count}개")
        print(f"   실패: {failed_count}개")
    
    else:
        # 선택적 삭제 예제
        print("\n선택적 문서 삭제")
        print("-" * 40)
        
        # 0. 현재 존재하는 문서들 먼저 확인
        print("\n0️⃣ 현재 존재하는 문서 확인")
        query = "SELECT c.id, c.category FROM c"
        all_docs = list(container.query_items(query=query, enable_cross_partition_query=True))
        print(f"   현재 문서 수: {len(all_docs)}개")
        for doc in all_docs:
            print(f"   • ID: {doc['id']}, Category: {doc.get('category', 'N/A')}")
        
        # 1. 특정 ID 삭제 - 안전한 삭제 방식
        print("\n1️⃣ 특정 ID(doc3) 삭제 시도")
        target_docs = [doc for doc in all_docs if doc['id'] == 'doc3']
        
        if target_docs:
            doc3 = target_docs[0]
            partition_key_value = doc3.get('category')
            print(f"   찾은 문서: ID={doc3['id']}, 파티션 키={partition_key_value}")
            
            try:
                # ETag를 사용한 안전한 삭제 (옵션)
                container.delete_item(item=doc3['id'], partition_key=partition_key_value)
                print("   ✅ doc3 삭제 완료")
            except exceptions.CosmosResourceNotFoundError:
                print("   ℹ️  doc3가 이미 삭제되었거나 존재하지 않습니다.")
            except Exception as e:
                print(f"   ❌ 삭제 실패: {e}")
        else:
            print("   ℹ️  doc3 문서가 현재 존재하지 않습니다.")
        
        # 2. 새로운 테스트 문서 생성 후 삭제
        print("\n2️⃣ 테스트 문서 생성 후 삭제")
        test_doc = {
            "id": "test_delete_doc",
            "category": "test",
            "message": "삭제 테스트용 문서",
            "created_at": datetime.now().isoformat()
        }
        
        try:
            # 테스트 문서 생성
            created_doc = container.create_item(body=test_doc)
            print(f"   ✅ 테스트 문서 생성: ID={created_doc['id']}")
            
            # 즉시 삭제
            container.delete_item(item=created_doc['id'], partition_key=created_doc['category'])
            print(f"   ✅ 테스트 문서 삭제 완료")
            
        except exceptions.CosmosResourceExistsError:
            print("   ℹ️  테스트 문서가 이미 존재합니다. 삭제 시도...")
            try:
                container.delete_item(item=test_doc['id'], partition_key=test_doc['category'])
                print("   ✅ 기존 테스트 문서 삭제 완료")
            except exceptions.CosmosResourceNotFoundError:
                print("   ℹ️  테스트 문서가 이미 삭제되었습니다.")
        except Exception as e:
            print(f"   ❌ 테스트 문서 작업 실패: {e}")
        
        # 3. 특정 카테고리의 모든 문서 삭제
        print("\n3️⃣ pipeline_log 카테고리 문서 삭제")
        pipeline_docs = [doc for doc in all_docs if doc.get('category') == 'pipeline_log']
        
        if pipeline_docs:
            for doc in pipeline_docs:
                try:
                    container.delete_item(item=doc['id'], partition_key=doc['category'])
                    print(f"   ✅ 삭제됨: ID={doc['id']}")
                except exceptions.CosmosResourceNotFoundError:
                    print(f"   ℹ️  이미 삭제됨: ID={doc['id']}")
                except Exception as e:
                    print(f"   ❌ 삭제 실패: ID={doc['id']}, 오류={e}")
        else:
            print("   삭제할 pipeline_log 문서가 없습니다.")
    
    # 삭제 후 남은 문서 수 확인
    print("\n📊 현재 컨테이너 상태:")
    query = "SELECT VALUE COUNT(1) FROM c"
    result = list(container.query_items(query=query, enable_cross_partition_query=True))
    remaining_count = result[0] if result else 0
    print(f"   남은 문서 수: {remaining_count}개")
    
    # 남은 문서 목록 표시
    if remaining_count > 0 and remaining_count <= 10:
        print("\n   남은 문서 목록:")
        query = "SELECT c.id, c.category FROM c"
        remaining_docs = list(container.query_items(query=query, enable_cross_partition_query=True))
        for doc in remaining_docs:
            print(f"   • ID: {doc['id']}, Category: {doc.get('category', 'N/A')}")

# 선택적 삭제 실행 (특정 문서만 삭제)
delete_data(container, delete_all=False)

# 모든 문서를 삭제하려면 아래 주석 해제
# delete_data(container, delete_all=True)

## 6. PostgreSQL 연결 및 CRUD 작업

In [ ]:
# PostgreSQL 설정 로드
PG_HOST = os.getenv('PG_HOST')
PG_PORT = os.getenv('PG_PORT', '5432')
PG_DATABASE = os.getenv('PG_DATABASE')
PG_USER = os.getenv('PG_USER')
PG_PASSWORD = os.getenv('PG_PASSWORD')

print(f"PostgreSQL 연결 정보:")
print(f"   Host: {PG_HOST}")
print(f"   Port: {PG_PORT}")
print(f"   Database: {PG_DATABASE}")
print(f"   User: {PG_USER}")
print(f"   Password: {'*' * 10 if PG_PASSWORD else 'Not set'}")

# SQLAlchemy 연결 문자열 생성
POSTGRES_URL = f"postgresql://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DATABASE}"

print(f"\n✅ PostgreSQL 설정 로드 완료")

In [50]:
def test_postgresql_connection_and_create_tables():
    """PostgreSQL 연결 테스트 및 테이블 생성"""
    
    print("=" * 60)
    print("PostgreSQL 연결 테스트 및 테이블 생성")
    print("=" * 60)
    
    try:
        # SQLAlchemy 엔진 생성
        engine = create_engine(POSTGRES_URL)
        
        # 연결 테스트
        with engine.connect() as conn:
            # 기본 연결 테스트
            result = conn.execute(text("SELECT version();"))
            version = result.fetchone()[0]
            print(f"✅ PostgreSQL 연결 성공!")
            print(f"   버전: {version}")
            
            # 현재 데이터베이스 정보
            result = conn.execute(text("SELECT current_database(), current_user;"))
            db_info = result.fetchone()
            print(f"   현재 DB: {db_info[0]}")
            print(f"   사용자: {db_info[1]}")
            
            # 기존 테이블 확인
            result = conn.execute(text("""
                SELECT table_name 
                FROM information_schema.tables 
                WHERE table_schema = 'public' 
                AND table_type = 'BASE TABLE'
                ORDER BY table_name;
            """))
            existing_tables = [row[0] for row in result.fetchall()]
            print(f"\n📊 기존 테이블 수: {len(existing_tables)}개")
            if existing_tables:
                for table in existing_tables:
                    print(f"   • {table}")
        
        # 샘플 테이블 생성
        print(f"\n🏗️  샘플 테이블 생성")
        print("-" * 40)
        
        with engine.connect() as conn:
            # 트랜잭션 시작
            trans = conn.begin()
            
            try:
                # 1. 고객 정보 테이블
                conn.execute(text("""
                    CREATE TABLE IF NOT EXISTS sample_customers (
                        customer_id SERIAL PRIMARY KEY,
                        name VARCHAR(100) NOT NULL,
                        email VARCHAR(150) UNIQUE NOT NULL,
                        phone VARCHAR(20),
                        registration_date DATE DEFAULT CURRENT_DATE,
                        is_active BOOLEAN DEFAULT true,
                        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                        updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                    );
                """))
                print("   ✅ sample_customers 테이블 생성")
                
                # 2. 주문 정보 테이블
                conn.execute(text("""
                    CREATE TABLE IF NOT EXISTS sample_orders (
                        order_id SERIAL PRIMARY KEY,
                        customer_id INTEGER REFERENCES sample_customers(customer_id),
                        order_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                        total_amount DECIMAL(10,2) NOT NULL,
                        status VARCHAR(20) DEFAULT 'pending',
                        shipping_address TEXT,
                        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                        updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                    );
                """))
                print("   ✅ sample_orders 테이블 생성")
                
                # 3. 상품 정보 테이블
                conn.execute(text("""
                    CREATE TABLE IF NOT EXISTS sample_products (
                        product_id SERIAL PRIMARY KEY,
                        name VARCHAR(200) NOT NULL,
                        description TEXT,
                        price DECIMAL(10,2) NOT NULL,
                        category VARCHAR(50),
                        stock_quantity INTEGER DEFAULT 0,
                        is_available BOOLEAN DEFAULT true,
                        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                        updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                    );
                """))
                print("   ✅ sample_products 테이블 생성")
                
                # 4. 메타데이터 테이블 (Rubicon 프로젝트용)
                conn.execute(text("""
                    CREATE TABLE IF NOT EXISTS rubicon_metadata (
                        metadata_id SERIAL PRIMARY KEY,
                        table_name VARCHAR(100) NOT NULL,
                        table_description TEXT,
                        column_info JSONB,
                        data_profile JSONB,
                        tags TEXT[],
                        created_by VARCHAR(50),
                        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                        updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                    );
                """))
                print("   ✅ rubicon_metadata 테이블 생성")
                
                # 인덱스 생성
                conn.execute(text("CREATE INDEX IF NOT EXISTS idx_customers_email ON sample_customers(email);"))
                conn.execute(text("CREATE INDEX IF NOT EXISTS idx_orders_customer_id ON sample_orders(customer_id);"))
                conn.execute(text("CREATE INDEX IF NOT EXISTS idx_orders_date ON sample_orders(order_date);"))
                conn.execute(text("CREATE INDEX IF NOT EXISTS idx_products_category ON sample_products(category);"))
                conn.execute(text("CREATE INDEX IF NOT EXISTS idx_metadata_table_name ON rubicon_metadata(table_name);"))
                print("   ✅ 인덱스 생성 완료")
                
                # 트랜잭션 커밋
                trans.commit()
                print("\n✅ 모든 테이블 생성 완료!")
                
            except Exception as e:
                trans.rollback()
                print(f"❌ 테이블 생성 실패: {e}")
                return None
        
        return engine
        
    except Exception as e:
        print(f"❌ PostgreSQL 연결 실패: {e}")
        return None

# PostgreSQL 연결 및 테이블 생성
pg_engine = test_postgresql_connection_and_create_tables()

PostgreSQL 연결 테스트 및 테이블 생성
✅ PostgreSQL 연결 성공!
   버전: PostgreSQL 17.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 11.2.0, 64-bit
   현재 DB: postgres
   사용자: postgresql

📊 기존 테이블 수: 0개

🏗️  샘플 테이블 생성
----------------------------------------
   ✅ sample_customers 테이블 생성
   ✅ sample_orders 테이블 생성
   ✅ sample_products 테이블 생성
   ✅ rubicon_metadata 테이블 생성
   ✅ 인덱스 생성 완료

✅ 모든 테이블 생성 완료!


In [51]:
def insert_sample_data_postgresql(engine):
    """PostgreSQL에 샘플 데이터 삽입"""
    
    print("=" * 60)
    print("PostgreSQL 샘플 데이터 삽입")
    print("=" * 60)
    
    if not engine:
        print("❌ PostgreSQL 엔진이 초기화되지 않았습니다.")
        return
    
    try:
        with engine.connect() as conn:
            trans = conn.begin()
            
            try:
                # 기존 데이터 확인 및 정리
                print("1️⃣ 기존 데이터 정리")
                print("-" * 30)
                
                # 외래키 제약으로 인해 순서대로 삭제
                conn.execute(text("TRUNCATE TABLE sample_orders CASCADE;"))
                conn.execute(text("TRUNCATE TABLE sample_customers CASCADE;"))
                conn.execute(text("TRUNCATE TABLE sample_products CASCADE;"))
                conn.execute(text("TRUNCATE TABLE rubicon_metadata CASCADE;"))
                print("   ✅ 기존 데이터 정리 완료")
                
                # 2. 고객 데이터 삽입
                print("\n2️⃣ 고객 데이터 삽입")
                print("-" * 30)
                
                customers_data = [
                    ("김철수", "kim.cs@example.com", "010-1234-5678"),
                    ("이영희", "lee.yh@example.com", "010-2345-6789"),
                    ("박민수", "park.ms@example.com", "010-3456-7890"),
                    ("정은경", "jung.ek@example.com", "010-4567-8901"),
                    ("최동욱", "choi.dw@example.com", "010-5678-9012")
                ]
                
                for name, email, phone in customers_data:
                    result = conn.execute(text("""
                        INSERT INTO sample_customers (name, email, phone) 
                        VALUES (:name, :email, :phone) 
                        RETURNING customer_id;
                    """), {"name": name, "email": email, "phone": phone})
                    customer_id = result.fetchone()[0]
                    print(f"   ✅ 고객 추가: {name} (ID: {customer_id})")
                
                # 3. 상품 데이터 삽입
                print("\n3️⃣ 상품 데이터 삽입")
                print("-" * 30)
                
                products_data = [
                    ("노트북", "고성능 업무용 노트북", 1500000.00, "전자제품", 50),
                    ("무선 마우스", "블루투스 무선 마우스", 35000.00, "컴퓨터액세서리", 100),
                    ("키보드", "기계식 키보드", 120000.00, "컴퓨터액세서리", 75),
                    ("모니터", "27인치 4K 모니터", 450000.00, "전자제품", 30),
                    ("스마트폰", "최신 스마트폰", 900000.00, "전자제품", 25),
                    ("책상", "높이조절 책상", 300000.00, "가구", 20),
                    ("의자", "인체공학 의자", 250000.00, "가구", 15)
                ]
                
                for name, desc, price, category, stock in products_data:
                    result = conn.execute(text("""
                        INSERT INTO sample_products (name, description, price, category, stock_quantity) 
                        VALUES (:name, :desc, :price, :category, :stock)
                        RETURNING product_id;
                    """), {"name": name, "desc": desc, "price": price, "category": category, "stock": stock})
                    product_id = result.fetchone()[0]
                    print(f"   ✅ 상품 추가: {name} (ID: {product_id})")
                
                # 4. 주문 데이터 삽입
                print("\n4️⃣ 주문 데이터 삽입")
                print("-" * 30)
                
                orders_data = [
                    (1, 1500000.00, "completed", "서울시 강남구"),
                    (2, 155000.00, "completed", "서울시 서초구"),
                    (3, 900000.00, "pending", "부산시 해운대구"),
                    (1, 750000.00, "shipped", "서울시 강남구"),
                    (4, 550000.00, "completed", "대구시 중구"),
                    (5, 570000.00, "pending", "광주시 서구")
                ]
                
                for customer_id, amount, status, address in orders_data:
                    result = conn.execute(text("""
                        INSERT INTO sample_orders (customer_id, total_amount, status, shipping_address) 
                        VALUES (:customer_id, :amount, :status, :address)
                        RETURNING order_id;
                    """), {"customer_id": customer_id, "amount": amount, "status": status, "address": address})
                    order_id = result.fetchone()[0]
                    print(f"   ✅ 주문 추가: 고객 {customer_id}, 금액 {amount:,}원 (주문 ID: {order_id})")
                
                # 5. 메타데이터 삽입 (Rubicon 프로젝트용)
                print("\n5️⃣ Rubicon 메타데이터 삽입")
                print("-" * 30)
                
                metadata_records = [
                    {
                        "table_name": "sample_customers",
                        "description": "샘플 고객 정보 테이블",
                        "column_info": {
                            "customer_id": {"type": "SERIAL", "description": "고객 고유 식별자"},
                            "name": {"type": "VARCHAR(100)", "description": "고객 이름"},
                            "email": {"type": "VARCHAR(150)", "description": "이메일 주소"},
                            "phone": {"type": "VARCHAR(20)", "description": "전화번호"}
                        },
                        "data_profile": {
                            "row_count": 5,
                            "null_percentage": 0.0,
                            "unique_emails": 5
                        },
                        "tags": ["customer", "personal_info", "master_data"],
                        "created_by": "system"
                    },
                    {
                        "table_name": "sample_orders",
                        "description": "샘플 주문 정보 테이블",
                        "column_info": {
                            "order_id": {"type": "SERIAL", "description": "주문 고유 식별자"},
                            "customer_id": {"type": "INTEGER", "description": "고객 ID (외래키)"},
                            "total_amount": {"type": "DECIMAL(10,2)", "description": "총 주문 금액"},
                            "status": {"type": "VARCHAR(20)", "description": "주문 상태"}
                        },
                        "data_profile": {
                            "row_count": 6,
                            "avg_amount": 720833.33,
                            "status_distribution": {"completed": 3, "pending": 2, "shipped": 1}
                        },
                        "tags": ["order", "transaction", "business"],
                        "created_by": "system"
                    }
                ]
                
                for metadata in metadata_records:
                    conn.execute(text("""
                        INSERT INTO rubicon_metadata 
                        (table_name, table_description, column_info, data_profile, tags, created_by) 
                        VALUES (:table_name, :description, :column_info, :data_profile, :tags, :created_by);
                    """), {
                        "table_name": metadata["table_name"],
                        "description": metadata["description"],
                        "column_info": json.dumps(metadata["column_info"]),
                        "data_profile": json.dumps(metadata["data_profile"]),
                        "tags": metadata["tags"],
                        "created_by": metadata["created_by"]
                    })
                    print(f"   ✅ 메타데이터 추가: {metadata['table_name']}")
                
                # 트랜잭션 커밋
                trans.commit()
                print("\n✅ 모든 샘플 데이터 삽입 완료!")
                
                # 삽입 결과 요약
                print("\n📊 삽입 결과 요약:")
                result = conn.execute(text("SELECT COUNT(*) FROM sample_customers;"))
                print(f"   고객: {result.fetchone()[0]}명")
                
                result = conn.execute(text("SELECT COUNT(*) FROM sample_products;"))
                print(f"   상품: {result.fetchone()[0]}개")
                
                result = conn.execute(text("SELECT COUNT(*) FROM sample_orders;"))
                print(f"   주문: {result.fetchone()[0]}건")
                
                result = conn.execute(text("SELECT COUNT(*) FROM rubicon_metadata;"))
                print(f"   메타데이터: {result.fetchone()[0]}건")
                
            except Exception as e:
                trans.rollback()
                print(f"❌ 데이터 삽입 실패: {e}")
                
    except Exception as e:
        print(f"❌ 연결 오류: {e}")

# 샘플 데이터 삽입 실행
insert_sample_data_postgresql(pg_engine)

PostgreSQL 샘플 데이터 삽입
1️⃣ 기존 데이터 정리
------------------------------
   ✅ 기존 데이터 정리 완료

2️⃣ 고객 데이터 삽입
------------------------------
   ✅ 고객 추가: 김철수 (ID: 1)
   ✅ 고객 추가: 이영희 (ID: 2)
   ✅ 고객 추가: 박민수 (ID: 3)
   ✅ 고객 추가: 정은경 (ID: 4)
   ✅ 고객 추가: 최동욱 (ID: 5)

3️⃣ 상품 데이터 삽입
------------------------------
   ✅ 상품 추가: 노트북 (ID: 1)
   ✅ 상품 추가: 무선 마우스 (ID: 2)
   ✅ 상품 추가: 키보드 (ID: 3)
   ✅ 상품 추가: 모니터 (ID: 4)
   ✅ 상품 추가: 스마트폰 (ID: 5)
   ✅ 상품 추가: 책상 (ID: 6)
   ✅ 상품 추가: 의자 (ID: 7)

4️⃣ 주문 데이터 삽입
------------------------------
   ✅ 주문 추가: 고객 1, 금액 1,500,000.0원 (주문 ID: 1)
   ✅ 주문 추가: 고객 2, 금액 155,000.0원 (주문 ID: 2)
   ✅ 주문 추가: 고객 3, 금액 900,000.0원 (주문 ID: 3)
   ✅ 주문 추가: 고객 1, 금액 750,000.0원 (주문 ID: 4)
   ✅ 주문 추가: 고객 4, 금액 550,000.0원 (주문 ID: 5)
   ✅ 주문 추가: 고객 5, 금액 570,000.0원 (주문 ID: 6)

5️⃣ Rubicon 메타데이터 삽입
------------------------------
   ✅ 메타데이터 추가: sample_customers
   ✅ 메타데이터 추가: sample_orders

✅ 모든 샘플 데이터 삽입 완료!

📊 삽입 결과 요약:
   고객: 5명
   상품: 7개
   주문: 6건
   메타데이터: 2건


In [52]:
def query_postgresql_data(engine):
    """PostgreSQL 데이터 조회 - 다양한 쿼리 예제"""
    
    print("=" * 60)
    print("PostgreSQL 데이터 조회")
    print("=" * 60)
    
    if not engine:
        print("❌ PostgreSQL 엔진이 초기화되지 않았습니다.")
        return
    
    try:
        with engine.connect() as conn:
            
            # 1. 기본 SELECT 쿼리
            print("1️⃣ 모든 고객 조회")
            print("-" * 40)
            result = conn.execute(text("SELECT customer_id, name, email, registration_date FROM sample_customers ORDER BY customer_id;"))
            customers = result.fetchall()
            print(f"   총 고객 수: {len(customers)}명")
            for customer in customers:
                print(f"   • ID: {customer[0]}, 이름: {customer[1]}, 이메일: {customer[2]}, 가입일: {customer[3]}")
            
            # 2. JOIN 쿼리
            print("\n2️⃣ 고객별 주문 현황 (JOIN)")
            print("-" * 40)
            result = conn.execute(text("""
                SELECT 
                    c.name, 
                    c.email, 
                    COUNT(o.order_id) as order_count,
                    COALESCE(SUM(o.total_amount), 0) as total_spent
                FROM sample_customers c
                LEFT JOIN sample_orders o ON c.customer_id = o.customer_id
                GROUP BY c.customer_id, c.name, c.email
                ORDER BY total_spent DESC;
            """))
            customer_orders = result.fetchall()
            for customer in customer_orders:
                print(f"   • {customer[0]} ({customer[1]}): 주문 {customer[2]}건, 총 구매액 {customer[3]:,}원")
            
            # 3. 집계 함수 사용
            print("\n3️⃣ 주문 통계")
            print("-" * 40)
            result = conn.execute(text("""
                SELECT 
                    status,
                    COUNT(*) as count,
                    AVG(total_amount) as avg_amount,
                    SUM(total_amount) as total_amount
                FROM sample_orders 
                GROUP BY status
                ORDER BY count DESC;
            """))
            order_stats = result.fetchall()
            for stat in order_stats:
                print(f"   • {stat[0]}: {stat[1]}건, 평균 {stat[2]:,.0f}원, 총액 {stat[3]:,}원")
            
            # 4. 조건부 쿼리
            print("\n4️⃣ 고액 주문 조회 (50만원 이상)")
            print("-" * 40)
            result = conn.execute(text("""
                SELECT 
                    o.order_id,
                    c.name,
                    o.total_amount,
                    o.status,
                    o.order_date
                FROM sample_orders o
                JOIN sample_customers c ON o.customer_id = c.customer_id
                WHERE o.total_amount >= 500000
                ORDER BY o.total_amount DESC;
            """))
            high_orders = result.fetchall()
            for order in high_orders:
                print(f"   • 주문 #{order[0]}: {order[1]}, {order[2]:,}원, {order[3]}, {order[4].strftime('%Y-%m-%d %H:%M')}")
            
            # 5. 상품 카테고리별 통계
            print("\n5️⃣ 상품 카테고리별 통계")
            print("-" * 40)
            result = conn.execute(text("""
                SELECT 
                    category,
                    COUNT(*) as product_count,
                    AVG(price) as avg_price,
                    SUM(stock_quantity) as total_stock
                FROM sample_products 
                GROUP BY category
                ORDER BY avg_price DESC;
            """))
            category_stats = result.fetchall()
            for stat in category_stats:
                print(f"   • {stat[0]}: {stat[1]}개 상품, 평균가격 {stat[2]:,.0f}원, 총 재고 {stat[3]}개")
            
            # 6. 서브쿼리 사용
            print("\n6️⃣ 가장 비싼 상품을 주문한 고객")
            print("-" * 40)
            result = conn.execute(text("""
                SELECT DISTINCT
                    c.name,
                    c.email,
                    o.total_amount
                FROM sample_customers c
                JOIN sample_orders o ON c.customer_id = o.customer_id
                WHERE o.total_amount = (SELECT MAX(total_amount) FROM sample_orders);
            """))
            top_customer = result.fetchall()
            for customer in top_customer:
                print(f"   • {customer[0]} ({customer[1]}): {customer[2]:,}원")
            
            # 7. 날짜 함수 사용
            print("\n7️⃣ 최근 주문 내역 (최근 30일)")
            print("-" * 40)
            result = conn.execute(text("""
                SELECT 
                    c.name,
                    o.total_amount,
                    o.status,
                    EXTRACT(DAY FROM AGE(CURRENT_TIMESTAMP, o.order_date)) as days_ago
                FROM sample_orders o
                JOIN sample_customers c ON o.customer_id = c.customer_id
                WHERE o.order_date >= CURRENT_TIMESTAMP - INTERVAL '30 days'
                ORDER BY o.order_date DESC;
            """))
            recent_orders = result.fetchall()
            if recent_orders:
                for order in recent_orders:
                    print(f"   • {order[0]}: {order[1]:,}원, {order[2]}, {int(order[3])}일 전")
            else:
                print("   최근 30일 내 주문이 없습니다.")
            
            # 8. JSON 데이터 쿼리 (Rubicon 메타데이터)
            print("\n8️⃣ Rubicon 메타데이터 조회")
            print("-" * 40)
            result = conn.execute(text("""
                SELECT 
                    table_name,
                    table_description,
                    data_profile->>'row_count' as row_count,
                    array_to_string(tags, ', ') as tags
                FROM rubicon_metadata
                ORDER BY table_name;
            """))
            metadata_info = result.fetchall()
            for meta in metadata_info:
                print(f"   • {meta[0]}: {meta[1]}")
                print(f"     행 수: {meta[2]}, 태그: {meta[3]}")
            
            # 9. 윈도우 함수 사용
            print("\n9️⃣ 고객별 주문 순위")
            print("-" * 40)
            result = conn.execute(text("""
                SELECT 
                    c.name,
                    o.total_amount,
                    o.order_date,
                    ROW_NUMBER() OVER (PARTITION BY c.customer_id ORDER BY o.total_amount DESC) as order_rank
                FROM sample_orders o
                JOIN sample_customers c ON o.customer_id = c.customer_id
                ORDER BY c.name, order_rank;
            """))
            ranked_orders = result.fetchall()
            current_customer = None
            for order in ranked_orders:
                if current_customer != order[0]:
                    current_customer = order[0]
                    print(f"   📋 {current_customer}:")
                print(f"     {order[3]}위: {order[1]:,}원 ({order[2].strftime('%Y-%m-%d')})")
                
    except Exception as e:
        print(f"❌ 쿼리 실행 오류: {e}")

# 데이터 조회 실행
query_postgresql_data(pg_engine)

PostgreSQL 데이터 조회
1️⃣ 모든 고객 조회
----------------------------------------
   총 고객 수: 5명
   • ID: 1, 이름: 김철수, 이메일: kim.cs@example.com, 가입일: 2025-09-15
   • ID: 2, 이름: 이영희, 이메일: lee.yh@example.com, 가입일: 2025-09-15
   • ID: 3, 이름: 박민수, 이메일: park.ms@example.com, 가입일: 2025-09-15
   • ID: 4, 이름: 정은경, 이메일: jung.ek@example.com, 가입일: 2025-09-15
   • ID: 5, 이름: 최동욱, 이메일: choi.dw@example.com, 가입일: 2025-09-15

2️⃣ 고객별 주문 현황 (JOIN)
----------------------------------------
   • 김철수 (kim.cs@example.com): 주문 2건, 총 구매액 2,250,000.00원
   • 박민수 (park.ms@example.com): 주문 1건, 총 구매액 900,000.00원
   • 최동욱 (choi.dw@example.com): 주문 1건, 총 구매액 570,000.00원
   • 정은경 (jung.ek@example.com): 주문 1건, 총 구매액 550,000.00원
   • 이영희 (lee.yh@example.com): 주문 1건, 총 구매액 155,000.00원

3️⃣ 주문 통계
----------------------------------------
   • completed: 3건, 평균 735,000원, 총액 2,205,000.00원
   • pending: 2건, 평균 735,000원, 총액 1,470,000.00원
   • shipped: 1건, 평균 750,000원, 총액 750,000.00원

4️⃣ 고액 주문 조회 (50만원 이상)
---------------------------------

In [54]:
def update_postgresql_data(engine):
    """PostgreSQL 데이터 수정 (UPDATE) 예제"""
    
    print("=" * 60)
    print("PostgreSQL 데이터 수정")
    print("=" * 60)
    
    if not engine:
        print("❌ PostgreSQL 엔진이 초기화되지 않았습니다.")
        return
    
    try:
        with engine.connect() as conn:
            trans = conn.begin()
            
            try:
                # 1. 단일 레코드 업데이트
                print("1️⃣ 고객 정보 업데이트")
                print("-" * 40)
                
                # 김철수의 전화번호 변경
                result = conn.execute(text("""
                    UPDATE sample_customers 
                    SET phone = '010-9999-0000', updated_at = CURRENT_TIMESTAMP 
                    WHERE name = '김철수'
                    RETURNING customer_id, name, phone;
                """))
                updated_customer = result.fetchone()
                if updated_customer:
                    print(f"   ✅ 고객 정보 업데이트: {updated_customer[1]} (ID: {updated_customer[0]})")
                    print(f"      새 전화번호: {updated_customer[2]}")
                
                # 2. 조건부 대량 업데이트
                print("\\n2️⃣ 주문 상태 대량 업데이트")
                print("-" * 40)
                
                # pending 상태의 주문을 processing으로 변경
                result = conn.execute(text("""
                    UPDATE sample_orders 
                    SET status = 'processing', updated_at = CURRENT_TIMESTAMP 
                    WHERE status = 'pending'
                    RETURNING order_id, status;
                """))
                updated_orders = result.fetchall()
                print(f"   ✅ {len(updated_orders)}개 주문 상태 업데이트:")
                for order in updated_orders:
                    print(f"      주문 #{order[0]}: {order[1]}")
                
                # 3. 계산된 값으로 업데이트
                print("\\n3️⃣ 상품 가격 인상 (10%)")
                print("-" * 40)
                
                # 전자제품 카테고리 가격 10% 인상
                result = conn.execute(text("""
                    UPDATE sample_products 
                    SET price = price * 1.1, updated_at = CURRENT_TIMESTAMP 
                    WHERE category = '전자제품'
                    RETURNING product_id, name, price;
                """))
                updated_products = result.fetchall()
                print(f"   ✅ {len(updated_products)}개 전자제품 가격 인상:")
                for product in updated_products:
                    print(f"      {product[1]}: {product[2]:,.0f}원")
                
                # 4. JOIN을 사용한 복잡한 업데이트
                print("\\n4️⃣ 고객 활성화 상태 업데이트 (주문 기준)")
                print("-" * 40)
                
                # 주문이 있는 고객을 활성 상태로, 없는 고객을 비활성 상태로 설정
                result = conn.execute(text("""
                    UPDATE sample_customers 
                    SET is_active = CASE 
                        WHEN EXISTS (SELECT 1 FROM sample_orders WHERE sample_orders.customer_id = sample_customers.customer_id) 
                        THEN true 
                        ELSE false 
                    END,
                    updated_at = CURRENT_TIMESTAMP
                    RETURNING customer_id, name, is_active;
                """))
                updated_customers = result.fetchall()
                print(f"   ✅ {len(updated_customers)}명 고객 활성화 상태 업데이트:")
                for customer in updated_customers:
                    status = "활성" if customer[2] else "비활성"
                    print(f"      {customer[1]} (ID: {customer[0]}): {status}")
                
                # 5. 메타데이터 업데이트 (JSON 필드)
                print("\\n5️⃣ 메타데이터 프로파일 업데이트")
                print("-" * 40)
                
                # 실제 데이터 개수로 메타데이터 업데이트
                customer_count_result = conn.execute(text("SELECT COUNT(*) FROM sample_customers;"))
                customer_count = customer_count_result.fetchone()[0]
                
                order_count_result = conn.execute(text("SELECT COUNT(*) FROM sample_orders;"))
                order_count = order_count_result.fetchone()[0]
                
                # 메타데이터 업데이트
                conn.execute(text("""
                    UPDATE rubicon_metadata 
                    SET data_profile = jsonb_set(data_profile, '{row_count}', :count::text::jsonb),
                        updated_at = CURRENT_TIMESTAMP
                    WHERE table_name = 'sample_customers';
                """), {"count": str(customer_count)})
                
                conn.execute(text("""
                    UPDATE rubicon_metadata 
                    SET data_profile = jsonb_set(data_profile, '{row_count}', :count::text::jsonb),
                        updated_at = CURRENT_TIMESTAMP
                    WHERE table_name = 'sample_orders';
                """), {"count": str(order_count)})
                
                print(f"   ✅ 메타데이터 업데이트 완료")
                print(f"      sample_customers 행 수: {customer_count}")
                print(f"      sample_orders 행 수: {order_count}")
                
                # 6. 배치 업데이트 (여러 레코드를 한 번에)
                print("\\n6️⃣ 재고 조정 (배치 업데이트)")
                print("-" * 40)
                
                # 재고가 부족한 상품들의 재고를 100개로 설정
                result = conn.execute(text("""
                    UPDATE sample_products 
                    SET stock_quantity = 100, updated_at = CURRENT_TIMESTAMP 
                    WHERE stock_quantity < 30
                    RETURNING product_id, name, stock_quantity;
                """))
                restocked_products = result.fetchall()
                if restocked_products:
                    print(f"   ✅ {len(restocked_products)}개 상품 재고 보충:")
                    for product in restocked_products:
                        print(f"      {product[1]}: 재고 {product[2]}개")
                else:
                    print("   재고 보충이 필요한 상품이 없습니다.")
                
                # 트랜잭션 커밋
                trans.commit()
                print("\\n✅ 모든 업데이트 완료!")
                
                # 업데이트 결과 요약
                print("\\n📊 업데이트 결과 요약:")
                
                # 최근 업데이트된 레코드 수 확인
                result = conn.execute(text("""
                    SELECT 'customers' as table_name, COUNT(*) as updated_count
                    FROM sample_customers 
                    WHERE updated_at > CURRENT_TIMESTAMP - INTERVAL '1 minute'
                    UNION ALL
                    SELECT 'orders', COUNT(*)
                    FROM sample_orders 
                    WHERE updated_at > CURRENT_TIMESTAMP - INTERVAL '1 minute'
                    UNION ALL
                    SELECT 'products', COUNT(*)
                    FROM sample_products 
                    WHERE updated_at > CURRENT_TIMESTAMP - INTERVAL '1 minute'
                    UNION ALL
                    SELECT 'metadata', COUNT(*)
                    FROM rubicon_metadata 
                    WHERE updated_at > CURRENT_TIMESTAMP - INTERVAL '1 minute';
                """))
                
                update_summary = result.fetchall()
                for summary in update_summary:
                    print(f"   {summary[0]}: {summary[1]}개 레코드 업데이트")
                
            except Exception as e:
                trans.rollback()
                print(f"❌ 업데이트 실패 (롤백됨): {e}")
                
    except Exception as e:
        print(f"❌ 연결 오류: {e}")

# 데이터 수정 실행
update_postgresql_data(pg_engine)

PostgreSQL 데이터 수정
1️⃣ 고객 정보 업데이트
----------------------------------------
   ✅ 고객 정보 업데이트: 김철수 (ID: 1)
      새 전화번호: 010-9999-0000
\n2️⃣ 주문 상태 대량 업데이트
----------------------------------------
   ✅ 2개 주문 상태 업데이트:
      주문 #3: processing
      주문 #6: processing
\n3️⃣ 상품 가격 인상 (10%)
----------------------------------------
   ✅ 3개 전자제품 가격 인상:
      노트북: 1,650,000원
      모니터: 495,000원
      스마트폰: 990,000원
\n4️⃣ 고객 활성화 상태 업데이트 (주문 기준)
----------------------------------------
   ✅ 5명 고객 활성화 상태 업데이트:
      이영희 (ID: 2): 활성
      박민수 (ID: 3): 활성
      정은경 (ID: 4): 활성
      최동욱 (ID: 5): 활성
      김철수 (ID: 1): 활성
\n5️⃣ 메타데이터 프로파일 업데이트
----------------------------------------
❌ 업데이트 실패 (롤백됨): (psycopg2.errors.SyntaxError) syntax error at or near ":"
LINE 3: ..._profile = jsonb_set(data_profile, '{row_count}', :count::te...
                                                             ^

[SQL: 
                    UPDATE rubicon_metadata 
                    SET data_profile = jsonb_set(data_profile,

In [55]:
def delete_postgresql_data(engine, delete_all=False):
    """PostgreSQL 데이터 삭제 (DELETE) 예제"""
    
    print("=" * 60)
    print("PostgreSQL 데이터 삭제")
    print("=" * 60)
    
    if not engine:
        print("❌ PostgreSQL 엔진이 초기화되지 않았습니다.")
        return
    
    try:
        with engine.connect() as conn:
            trans = conn.begin()
            
            try:
                if delete_all:
                    # 모든 데이터 삭제
                    print("⚠️  모든 데이터 삭제 작업")
                    print("-" * 40)
                    
                    # 외래키 제약으로 인해 순서대로 삭제
                    result = conn.execute(text("DELETE FROM sample_orders;"))
                    orders_deleted = result.rowcount
                    print(f"   ✅ 주문 데이터 삭제: {orders_deleted}건")
                    
                    result = conn.execute(text("DELETE FROM sample_customers;"))
                    customers_deleted = result.rowcount
                    print(f"   ✅ 고객 데이터 삭제: {customers_deleted}건")
                    
                    result = conn.execute(text("DELETE FROM sample_products;"))
                    products_deleted = result.rowcount
                    print(f"   ✅ 상품 데이터 삭제: {products_deleted}건")
                    
                    result = conn.execute(text("DELETE FROM rubicon_metadata;"))
                    metadata_deleted = result.rowcount
                    print(f"   ✅ 메타데이터 삭제: {metadata_deleted}건")
                    
                else:
                    # 선택적 삭제 예제들
                    print("선택적 데이터 삭제")
                    print("-" * 40)
                    
                    # 0. 현재 데이터 상태 확인
                    print("\\n0️⃣ 현재 데이터 상태 확인")
                    print("-" * 30)
                    result = conn.execute(text("SELECT COUNT(*) FROM sample_customers;"))
                    customer_count = result.fetchone()[0]
                    result = conn.execute(text("SELECT COUNT(*) FROM sample_orders;"))
                    order_count = result.fetchone()[0]
                    result = conn.execute(text("SELECT COUNT(*) FROM sample_products;"))
                    product_count = result.fetchone()[0]
                    
                    print(f"   고객: {customer_count}명, 주문: {order_count}건, 상품: {product_count}개")
                    
                    # 1. 단일 레코드 삭제
                    print("\\n1️⃣ 특정 고객 삭제")
                    print("-" * 30)
                    
                    # 주문이 없는 고객 찾기
                    result = conn.execute(text("""
                        SELECT c.customer_id, c.name 
                        FROM sample_customers c
                        LEFT JOIN sample_orders o ON c.customer_id = o.customer_id
                        WHERE o.customer_id IS NULL
                        LIMIT 1;
                    """))
                    customer_to_delete = result.fetchone()
                    
                    if customer_to_delete:
                        result = conn.execute(text("""
                            DELETE FROM sample_customers 
                            WHERE customer_id = :customer_id
                        """), {"customer_id": customer_to_delete[0]})
                        if result.rowcount > 0:
                            print(f"   ✅ 고객 삭제: {customer_to_delete[1]} (ID: {customer_to_delete[0]})")
                    else:
                        print("   모든 고객이 주문을 가지고 있어 삭제할 수 없습니다.")
                    
                    # 2. 조건부 대량 삭제
                    print("\\n2️⃣ 완료된 주문 중 오래된 것 삭제")
                    print("-" * 30)
                    
                    # 완료된 주문 중 가장 오래된 것 삭제
                    result = conn.execute(text("""
                        DELETE FROM sample_orders 
                        WHERE status = 'completed' 
                        AND order_date < (
                            SELECT MAX(order_date) - INTERVAL '1 hour' 
                            FROM sample_orders 
                            WHERE status = 'completed'
                        )
                        RETURNING order_id, total_amount;
                    """))
                    deleted_orders = result.fetchall()
                    if deleted_orders:
                        print(f"   ✅ {len(deleted_orders)}건의 오래된 주문 삭제:")
                        for order in deleted_orders:
                            print(f"      주문 #{order[0]}: {order[1]:,}원")
                    else:
                        print("   삭제할 오래된 주문이 없습니다.")
                    
                    # 3. 재고가 없는 상품 삭제
                    print("\\n3️⃣ 재고가 없는 상품 삭제")
                    print("-" * 30)
                    
                    # 먼저 재고 0인 상품이 있는지 확인하고 없으면 임시로 만들기
                    result = conn.execute(text("SELECT COUNT(*) FROM sample_products WHERE stock_quantity = 0;"))
                    zero_stock_count = result.fetchone()[0]
                    
                    if zero_stock_count == 0:
                        # 테스트를 위해 하나의 상품 재고를 0으로 설정
                        conn.execute(text("""
                            UPDATE sample_products 
                            SET stock_quantity = 0 
                            WHERE product_id = (SELECT MIN(product_id) FROM sample_products)
                        """))
                        print("   (테스트를 위해 하나의 상품 재고를 0으로 설정)")
                    
                    result = conn.execute(text("""
                        DELETE FROM sample_products 
                        WHERE stock_quantity = 0
                        RETURNING product_id, name;
                    """))
                    deleted_products = result.fetchall()
                    if deleted_products:
                        print(f"   ✅ {len(deleted_products)}개의 재고 없는 상품 삭제:")
                        for product in deleted_products:
                            print(f"      {product[1]} (ID: {product[0]})")
                    else:
                        print("   재고가 없는 상품이 없습니다.")
                    
                    # 4. CASCADE 삭제 테스트 (고객 삭제 시 관련 주문도 함께 삭제)
                    print("\\n4️⃣ 고객 및 관련 주문 함께 삭제")
                    print("-" * 30)
                    
                    # 주문이 가장 적은 고객 찾기
                    result = conn.execute(text("""
                        SELECT c.customer_id, c.name, COUNT(o.order_id) as order_count
                        FROM sample_customers c
                        LEFT JOIN sample_orders o ON c.customer_id = o.customer_id
                        GROUP BY c.customer_id, c.name
                        HAVING COUNT(o.order_id) > 0
                        ORDER BY COUNT(o.order_id)
                        LIMIT 1;
                    """))
                    customer_with_orders = result.fetchone()
                    
                    if customer_with_orders:
                        customer_id = customer_with_orders[0]
                        customer_name = customer_with_orders[1]
                        order_count = customer_with_orders[2]
                        
                        # 먼저 관련 주문 삭제
                        result = conn.execute(text("""
                            DELETE FROM sample_orders 
                            WHERE customer_id = :customer_id
                        """), {"customer_id": customer_id})
                        deleted_order_count = result.rowcount
                        
                        # 그 다음 고객 삭제
                        result = conn.execute(text("""
                            DELETE FROM sample_customers 
                            WHERE customer_id = :customer_id
                        """), {"customer_id": customer_id})
                        
                        if result.rowcount > 0:
                            print(f"   ✅ 고객 및 관련 데이터 삭제: {customer_name}")
                            print(f"      삭제된 주문: {deleted_order_count}건")
                    else:
                        print("   삭제할 고객이 없습니다.")
                    
                    # 5. 특정 기간의 데이터 삭제
                    print("\\n5️⃣ 특정 기간 주문 삭제 (테스트)")
                    print("-" * 30)
                    
                    # 가장 최근 주문의 일부를 삭제 (테스트 목적)
                    result = conn.execute(text("""
                        DELETE FROM sample_orders 
                        WHERE order_id IN (
                            SELECT order_id 
                            FROM sample_orders 
                            ORDER BY order_date DESC 
                            LIMIT 1
                        )
                        RETURNING order_id, order_date;
                    """))
                    deleted_recent_orders = result.fetchall()
                    if deleted_recent_orders:
                        for order in deleted_recent_orders:
                            print(f"   ✅ 최근 주문 삭제: #{order[0]} ({order[1].strftime('%Y-%m-%d %H:%M')})")
                    else:
                        print("   삭제할 주문이 없습니다.")
                    
                    # 6. 메타데이터 정리
                    print("\\n6️⃣ 메타데이터 정리")
                    print("-" * 30)
                    
                    # 존재하지 않는 테이블의 메타데이터 삭제 (예시)
                    result = conn.execute(text("""
                        DELETE FROM rubicon_metadata 
                        WHERE table_name NOT IN (
                            SELECT table_name 
                            FROM information_schema.tables 
                            WHERE table_schema = 'public'
                        )
                        RETURNING table_name;
                    """))
                    deleted_metadata = result.fetchall()
                    if deleted_metadata:
                        print(f"   ✅ 존재하지 않는 테이블의 메타데이터 삭제:")
                        for meta in deleted_metadata:
                            print(f"      {meta[0]}")
                    else:
                        print("   정리할 메타데이터가 없습니다.")
                
                # 트랜잭션 커밋
                trans.commit()
                print("\\n✅ 모든 삭제 작업 완료!")
                
                # 삭제 후 상태 확인
                print("\\n📊 삭제 후 데이터 상태:")
                
                result = conn.execute(text("SELECT COUNT(*) FROM sample_customers;"))
                remaining_customers = result.fetchone()[0]
                
                result = conn.execute(text("SELECT COUNT(*) FROM sample_orders;"))
                remaining_orders = result.fetchone()[0]
                
                result = conn.execute(text("SELECT COUNT(*) FROM sample_products;"))
                remaining_products = result.fetchone()[0]
                
                result = conn.execute(text("SELECT COUNT(*) FROM rubicon_metadata;"))
                remaining_metadata = result.fetchone()[0]
                
                print(f"   남은 고객: {remaining_customers}명")
                print(f"   남은 주문: {remaining_orders}건")
                print(f"   남은 상품: {remaining_products}개")
                print(f"   남은 메타데이터: {remaining_metadata}건")
                
            except Exception as e:
                trans.rollback()
                print(f"❌ 삭제 실패 (롤백됨): {e}")
                
    except Exception as e:
        print(f"❌ 연결 오류: {e}")

# 선택적 삭제 실행
delete_postgresql_data(pg_engine, delete_all=False)

# 모든 데이터를 삭제하려면 아래 주석 해제
# delete_postgresql_data(pg_engine, delete_all=True)

PostgreSQL 데이터 삭제
선택적 데이터 삭제
----------------------------------------
\n0️⃣ 현재 데이터 상태 확인
------------------------------
   고객: 5명, 주문: 6건, 상품: 7개
\n1️⃣ 특정 고객 삭제
------------------------------
   모든 고객이 주문을 가지고 있어 삭제할 수 없습니다.
\n2️⃣ 완료된 주문 중 오래된 것 삭제
------------------------------
   삭제할 오래된 주문이 없습니다.
\n3️⃣ 재고가 없는 상품 삭제
------------------------------
   (테스트를 위해 하나의 상품 재고를 0으로 설정)
   ✅ 1개의 재고 없는 상품 삭제:
      노트북 (ID: 1)
\n4️⃣ 고객 및 관련 주문 함께 삭제
------------------------------
   ✅ 고객 및 관련 데이터 삭제: 박민수
      삭제된 주문: 1건
\n5️⃣ 특정 기간 주문 삭제 (테스트)
------------------------------
   ✅ 최근 주문 삭제: #6 (2025-09-15 07:58)
\n6️⃣ 메타데이터 정리
------------------------------
   정리할 메타데이터가 없습니다.
\n✅ 모든 삭제 작업 완료!
\n📊 삭제 후 데이터 상태:
   남은 고객: 4명
   남은 주문: 4건
   남은 상품: 6개
   남은 메타데이터: 2건
